##spaCy NER Training

Install dependencies

In [1]:
!pip install spacy spacy-transformers
!python -m spacy download en_core_web_md

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.13-cp310-cp310-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp310-cp310-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.10-cp310-cp310-win_amd64.whl.metadata (2.5 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.1-cp310-cp310-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached blis-1.3.0-cp310-cp310-win_amd64.whl.metadata (7.6 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached click-8.3

Load and inspect dataset

In [6]:
import json

train_path = "../data/cv_ner_format/train.jsonl"
dev_path = "../data/cv_ner_format/dev.jsonl"

def preview_jsonl(path, n=3):
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= n:  
                break
            print(json.loads(line))
            print("---")

preview_jsonl(train_path)

{'text': 'Curriculum Vitae\nName : <NAME>\nEmail : <EMAIL>\nContact : <PHONE> -------------------------------------------------------------------------------------------------------------------------------------\nOBJECTIVE:\nTo excel career in an organization which provides a challenging work environment in a global\nscenario and allows as an individual to grow professionally and to serve the organization to the\nbest of my abilities and become professionally trained to handle critical situations and\nresponsibilities.\nACADEMIC CREDENTIALS:\nBACHELOR OF ENGINEERING in CIVIL ENGINEERING from Shri Dharmasthala Manjunatheshwara College of Engineering and Technology, Dharwad.\nAcademic Level University Name of\nInstitutePercentage Year B.E. Degree, CIVIL Engg. VTU,BELGAU\nMShri Dharmasthala\nManjunatheshwara\nCollege Of\nEngineering &\nTechnology(Dharwad)712015 (12th) C.I.S.C.E I.C.S.ESt. Xavier’s School ,\nBokaro60.752010 (10th) C.I.S.C.E I.C.S.ESt. Xavier’s School ,\nBokaro72.752008\nSO

Train spaCy model

In [4]:
# convert_jsonl_to_spacy_fixed.py
import json
import spacy
from spacy.tokens import DocBin
from pathlib import Path

def convert_jsonl_to_spacy(jsonl_path, spacy_path, nlp):
    """Convert JSONL format to spaCy binary format"""
    doc_bin = DocBin(attrs=["ENT_IOB", "ENT_TYPE"])
    
    count = 0
    invalid_spans = 0
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            try:
                data = json.loads(line)
                text = data['text']
                entities = data.get('entities', [])
                
                # Create a Doc object
                doc = nlp.make_doc(text)
                
                # Convert entities to spaCy format
                spans = []
                for start, end, label in entities:
                    # Ensure start and end are within bounds
                    start = max(0, min(start, len(text)))
                    end = max(start, min(end, len(text)))
                    
                    # Get the entity text
                    entity_text = text[start:end]
                    
                    # Trim whitespace from the entity text
                    entity_text_stripped = entity_text.strip()
                    
                    # If the stripped text is empty, skip this entity
                    if not entity_text_stripped:
                        invalid_spans += 1
                        continue
                    
                    # Find new start and end positions after trimming
                    # Find how many leading whitespace chars to skip
                    leading_ws = len(entity_text) - len(entity_text.lstrip())
                    trailing_ws = len(entity_text) - len(entity_text.rstrip())
                    
                    actual_start = start + leading_ws
                    actual_end = end - trailing_ws
                    
                    # Ensure we have valid positions
                    if actual_start >= actual_end or actual_start < 0 or actual_end > len(text):
                        invalid_spans += 1
                        continue
                    
                    # Try different alignment modes
                    span = None
                    for alignment_mode in ['contract', 'expand', 'strict']:
                        span = doc.char_span(actual_start, actual_end, label=label, alignment_mode=alignment_mode)
                        if span is not None:
                            # Verify the span text doesn't start or end with whitespace
                            span_text = span.text
                            if span_text and not span_text[0].isspace() and not span_text[-1].isspace():
                                break
                            else:
                                span = None
                    
                    if span is not None:
                        spans.append(span)
                    else:
                        invalid_spans += 1
                
                # Set entities on the doc
                doc.ents = spans
                doc_bin.add(doc)
                count += 1
                
            except Exception as e:
                print(f"Error processing line {count + 1}: {e}")
                continue
    
    # Save to .spacy file
    doc_bin.to_disk(spacy_path)
    print(f"Converted {jsonl_path} -> {spacy_path} ({count} documents, {invalid_spans} invalid spans skipped)")

# Load a blank English model for tokenization
nlp = spacy.blank("en")

# Convert train and dev files
print("Converting train.jsonl...")
convert_jsonl_to_spacy(
    "../data/cv_ner_format/train.jsonl",
    "../data/cv_ner_format/train.spacy",
    nlp
)

print("\nConverting dev.jsonl...")
convert_jsonl_to_spacy(
    "../data/cv_ner_format/dev.jsonl",
    "../data/cv_ner_format/dev.spacy",
    nlp
)
print("\nConverting test.jsonl...")
convert_jsonl_to_spacy(
    "../data/cv_ner_format/test.jsonl",
    "../data/cv_ner_format/test.spacy",
    nlp
)

Converting train.jsonl...
Converted ../data/cv_ner_format/train.jsonl -> ../data/cv_ner_format/train.spacy (360 documents, 0 invalid spans skipped)

Converting dev.jsonl...
Converted ../data/cv_ner_format/dev.jsonl -> ../data/cv_ner_format/dev.spacy (45 documents, 0 invalid spans skipped)

Converting test.jsonl...
Converted ../data/cv_ner_format/test.jsonl -> ../data/cv_ner_format/test.spacy (46 documents, 0 invalid spans skipped)


In [8]:
!python -m spacy train ./configs/spacy_config.cfg --output ./models/spacy_ner/

[i] Saving to output directory: models\spacy_ner
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Set annotations on update for: ['tok2vec']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00   1324.23    0.00    0.00    0.00    0.00
  0     200        123.18  16101.84    0.00    0.00    0.00    0.00
  1     400       1872.75   1394.50    0.00    0.00    0.00    0.00
  1     600         79.45   1090.99    6.52   60.00    3.45    0.07
  2     800      27173.80   1232.40    8.60   66.67    4.60    0.09
  2    1000        192.29   1000.73    6.59   75.00    3.45    0.07
  3    1200        178.22   1028.84   11.76   40.00    6.90    0.12
  3    1400       1535.80   1094

Load trained model

In [9]:
import spacy

spacy_model_path = "./models/spacy_ner/model-best"
nlp_spacy = spacy.load(spacy_model_path)

print("Loaded spaCy model:", nlp_spacy)


Loaded spaCy model: <spacy.lang.en.English object at 0x00000163DCC8E320>


Test model on one example

In [11]:
sample_text = "Responsible for safety, cabin procedures and assisting passengers."

doc = nlp_spacy(sample_text)

for ent in doc.ents:
    print(ent.text, ent.label_)


Evaluate spaCy model on test set

In [12]:

test_path = "../data/cv_ner_format/test.spacy"

!python -m spacy evaluate ./models/spacy_ner/model-best ../data/cv_ner_format/test.spacy --output ./evaluation/spacy_eval.json


[i] Using CPU

================================== Results ==================================

TOK     100.00
NER P   47.69 
NER R   25.83 
NER F   33.51 
SPEED   10240 


=============================== NER (per type) ===============================

                              P       R       F
SKILL: BUILDING           83.33   35.71   50.00
SKILL: KNOWLEDGE          55.56   22.73   32.26
SKILL: SUPERVISION         0.00    0.00    0.00
SKILL: PLANNING           36.36   28.57   32.00
SKILL: MATERIALS          37.50   23.08   28.57
SKILL: RESPONSIBILITIES   61.54   30.77   41.03
SKILL: EQUIPMENT          40.00   23.08   29.27

[+] Saved results to evaluation\spacy_eval.json


#SECTION 2 — HuggingFace NER

Install dependencies

In [9]:
!pip install transformers datasets seqeval

Load dataset using HuggingFace datasets

In [10]:
from datasets import Dataset, Features, Value, Sequence
import json

def load_and_convert_jsonl(file_path):
    """Load JSONL and convert entities to dict format"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if not line.strip():
                continue
            try:
                obj = json.loads(line)
                
                # Convert entities from list format [start, end, label] to dict format
                if 'entities' in obj:
                    converted_entities = []
                    for entity in obj['entities']:
                        if isinstance(entity, list) and len(entity) == 3:
                            converted_entities.append({
                                "start": int(entity[0]),
                                "end": int(entity[1]),
                                "label": str(entity[2])
                            })
                        elif isinstance(entity, dict):
                            converted_entities.append({
                                "start": int(entity["start"]),
                                "end": int(entity["end"]),
                                "label": str(entity["label"])
                            })
                    obj['entities'] = converted_entities
                else:
                    obj['entities'] = []
                    
                data.append(obj)
            except Exception as e:
                print(f"Error on line {i+1}: {e}")
                continue
    return data

# Load all data
print("Loading data...")
train_data = load_and_convert_jsonl("../data/cv_ner_format/train.jsonl")
dev_data = load_and_convert_jsonl("../data/cv_ner_format/dev.jsonl")
test_data = load_and_convert_jsonl("../data/cv_ner_format/test.jsonl")

print(f"Loaded {len(train_data)} train, {len(dev_data)} dev, {len(test_data)} test examples")

# Verify conversion worked
if train_data:
    first_entity = train_data[0]['entities'][0] if train_data[0].get('entities') else None
    print(f"First entity type: {type(first_entity)}")
    print(f"Is dict: {isinstance(first_entity, dict) if first_entity else None}")

# Create datasets WITHOUT features first - let datasets infer the schema
print("\nCreating datasets (without explicit features)...")
train_ds = Dataset.from_list(train_data)
dev_ds = Dataset.from_list(dev_data)
test_ds = Dataset.from_list(test_data)

print("Datasets created successfully!")

# Now cast to the desired features
print("\nCasting to explicit features...")
features = Features({
    "text": Value("string"),
    "entities": Sequence({
        "start": Value("int32"),
        "end": Value("int32"),
        "label": Value("string")
    })
})

try:
    train_ds = train_ds.cast(features)
    dev_ds = dev_ds.cast(features)
    test_ds = test_ds.cast(features)
    print("Features cast successfully!")
except Exception as e:
    print(f"Warning: Could not cast features: {e}")
    print("Continuing without explicit features cast...")

train_ds, dev_ds, test_ds

Loading data...
Loaded 360 train, 45 dev, 46 test examples
First entity type: <class 'dict'>
Is dict: True

Creating datasets (without explicit features)...
Datasets created successfully!

Casting to explicit features...


Casting the dataset:   0%|          | 0/360 [00:00<?, ? examples/s]

list<item: struct<end: int64, label: string, start: int64>>
to
{'start': List(Value('int32')), 'end': List(Value('int32')), 'label': List(Value('string'))}
Continuing without explicit features cast...


(Dataset({
     features: ['text', 'entities'],
     num_rows: 360
 }),
 Dataset({
     features: ['text', 'entities'],
     num_rows: 45
 }),
 Dataset({
     features: ['text', 'entities'],
     num_rows: 46
 }))

Load label list

In [11]:
labels = [line.strip() for line in open("./configs/labels.txt")]
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

labels[:10], label2id

(['SKILL',
  'SKILL: BUILDING',
  'SKILL: KNOWLEDGE',
  'SKILL: SUPERVISION',
  'SKILL: PLANNING',
  'SKILL: MATERIALS',
  'SKILL: EQUIPMENT',
  'SKILL: RESPONSIBILITIES',
  '',
  'AVIATION: ROLE'],
 {'SKILL': 0,
  'SKILL: BUILDING': 1,
  'SKILL: KNOWLEDGE': 2,
  'SKILL: SUPERVISION': 3,
  'SKILL: PLANNING': 4,
  'SKILL: MATERIALS': 5,
  'SKILL: EQUIPMENT': 6,
  'SKILL: RESPONSIBILITIES': 7,
  '': 8,
  'AVIATION: ROLE': 9,
  'AVIATION: TRAINING': 10,
  'AVIATION: CERTIFICATION': 11,
  'AVIATION: AIRCRAFT': 12,
  'AVIATION: EXPERIENCE': 13,
  'AVIATION: SAFETY': 14,
  'AVIATION: LANGUAGE': 15})

Helper: Convert NER spans to token-level labels

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_and_align_labels(example):
    text = example["text"]
    entities = example["entities"]

    # Tokenize the text
    tokenized = tokenizer(text, truncation=True, max_length=256, return_offsets_mapping=True)

    # Initialize labels with -100 (ignored by loss) for all tokens
    # -100 is the default ignore_index for CrossEntropyLoss
    labels_out = [-100] * len(tokenized["input_ids"])

    # Align entity labels to tokens
    for entity in entities:
        # Handle dictionary format: {"start": int, "end": int, "label": str}
        start = entity["start"]
        end = entity["end"]
        label = entity["label"]
        
        # Find the token positions for the entity span
        first_token = tokenized.char_to_token(start)
        if first_token is None:
            continue
        last_token = tokenized.char_to_token(end - 1)
        if last_token is None:
            continue

        # Assign B- prefix to first token, I- prefix to subsequent tokens
        for i in range(first_token, last_token + 1):
            if i < len(labels_out):  # Ensure index is within bounds
                # Get label ID, default to "O" if label not found
                if i == first_token:
                    label_str = "B-" + label
                else:
                    label_str = "I-" + label
                
                # Map to label ID, use "O" (index 0) as fallback if not found
                label_id = label2id.get(label_str, label2id.get("O", 0))
                labels_out[i] = label_id

    # Convert labels to tensor-compatible format
    # Set special tokens (first [CLS] and last [SEP]) to -100 if they weren't touched
    # This ensures padding tokens are also -100
    example["input_ids"] = tokenized["input_ids"]
    example["attention_mask"] = tokenized["attention_mask"]
    example["labels"] = labels_out
    
    return example

train_dataset = train_ds.map(tokenize_and_align_labels)
dev_dataset = dev_ds.map(tokenize_and_align_labels)
test_dataset = test_ds.map(tokenize_and_align_labels)

Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

Load model

In [18]:
from transformers import AutoModelForTokenClassification

hf_model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define Trainer

In [22]:
!pip install transformers[torch]
!pip install accelerate>=0.26.0
!pip install seqeval

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./models/hf_ner/",
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Must match evaluation_strategy when using load_best_model_at_end
    logging_steps=20,
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=hf_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
)

C:\Users\21629\AppData\Local\Temp\ipykernel_28384\424407876.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Train HuggingFace model

In [ ]:
trainer.train()

Evaluate on test set

In [ ]:
results = trainer.evaluate(test_dataset)
results

Save evaluation results

In [ ]:
import json

with open("./evaluation/hf_eval.json", "w") as f:
    json.dump(results, f, indent=4)

#SECTION 3 — Comparing Models

In [ ]:
import json

with open("./evaluation/hf_eval.json") as f:
    hf = json.load(f)

with open("./evaluation/spacy_eval.json") as f:
    spacy = json.load(f)

hf, spacy
